In [1]:
import tensorflow as tf
import numpy as np

C:\Users\josep\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [3]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)

In [5]:
target = np.log(train.SalePrice)

In [6]:
numeric_features = train.select_dtypes(include=[np.number])
corr = numeric_features.corr()

print(corr['SalePrice'].sort_values(ascending=False)[:5])
print(corr['SalePrice'].sort_values(ascending=False)[-5:])

SalePrice      1.000000
OverallQual    0.790982
GrLivArea      0.708624
GarageCars     0.640409
GarageArea     0.623431
Name: SalePrice, dtype: float64
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64


In [7]:
nulls = pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns =['Null Count']
nulls.index.name = 'Feature'

print(nulls)

              Null Count
Feature                 
PoolQC              1453
MiscFeature         1406
Alley               1369
Fence               1179
FireplaceQu          690
LotFrontage          259
GarageCond            81
GarageType            81
GarageYrBlt           81
GarageFinish          81
GarageQual            81
BsmtExposure          38
BsmtFinType2          38
BsmtFinType1          37
BsmtCond              37
BsmtQual              37
MasVnrArea             8
MasVnrType             8
Electrical             1
Utilities              0
YearRemodAdd           0
MSSubClass             0
Foundation             0
ExterCond              0
ExterQual              0


In [8]:
train['enc_street'] = pd.get_dummies(train.Street, drop_first=True)
test['enc_street'] = pd.get_dummies(train.Street, drop_first=True)

In [9]:
def encode(x): 
    return 1 if x == 'Partial' else 0
train['enc_condition'] = train.SaleCondition.apply(encode)
test['enc_condition'] = test.SaleCondition.apply(encode)

In [10]:
data = train.select_dtypes(include=[np.number]).interpolate().dropna()

In [11]:
y = train.SalePrice
x = data.drop(['SalePrice','Id'], axis=1)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42,test_size=.33)


In [13]:

x1_train= MinMaxScaler(x_train)
y1_train= y_train
x1_test=x_test

In [14]:
y2_train = train.SalePrice
x2_train =  data.drop(['SalePrice','Id'], axis=1)

x3_train = MinMaxScaler(x_train)
y3_train = MinMaxScaler(y_train)


In [15]:
x3_train.shape

(978, 38)

In [16]:
x_test.shape

(482, 38)

In [17]:

learning_rate = 0.001
training_epochs = 15
batch_size = 100


# input place holders
X = tf.placeholder(tf.float32, None)
Y = tf.placeholder(tf.float32, None)

# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[38, 38],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([38]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[38, 38],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([38]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[38, 38],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([38]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[38, 38],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([38]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[38, 38],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(978):
        
        feed_dict = {X: x3_train, Y: y3_train, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



InvalidArgumentError: logits and labels must be broadcastable: logits_size=[978,38] labels_size=[1,978]
	 [[Node: softmax_cross_entropy_with_logits_sg = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](add_4, softmax_cross_entropy_with_logits_sg/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits_sg', defined at:
  File "C:\Users\josep\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\josep\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\josep\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\josep\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\josep\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\josep\Anaconda3\lib\site-pac

In [17]:
X = tf.placeholder(tf.float32, shape=None)
Y = tf.placeholder(tf.float32, shape=None)

# weights & bias for nn layers
W1 = tf.Variable(tf.random_normal([38,38]))
b1 = tf.Variable(tf.random_normal([38]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([38,38]))
b2 = tf.Variable(tf.random_normal([38]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([38,38]))
b3 = tf.Variable(tf.random_normal([38]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.Variable(tf.random_normal([38,38]))
b4 = tf.Variable(tf.random_normal([38]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.Variable(tf.random_normal([38,38]))
b5 = tf.Variable(tf.random_normal([38]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

w6 = tf.Variable(tf.random_normal([38,1]))
b6 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(L5, w6) + b6



In [24]:
learning_rate = 0.0001
training_epochs = 15
batch_size = 978

In [25]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [26]:
sess=tf.Session()
   # Initialize TensorFlow variables
sess.run(tf.global_variables_initializer())

In [27]:


for step in range(100001):
    sess.run(optimizer, feed_dict={X: x3_train, Y: y3_train})
    if step % 1000 == 0:
        print(step, sess.run(cost, feed_dict={X: x3_train, Y: y3_train}))
    


0 1667.7811
1000 1667.7811
2000 1667.7811
3000 1667.7811
4000 1667.7811


KeyboardInterrupt: 

In [ ]:

dict = sess.run(hypothesis, feed_dict={X:x_test})
print(dict)


In [ ]:
import keras


In [ ]:
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [ ]:
x3_train.shape

In [ ]:
model = Sequential([
    Dense(978, input_shape=(38,),activation='relu'),
    Dense(978,activation='relu'),
    Dense(1, activation='softmax')
])

moder.summary(

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [48]:
model.fit(x3_train,y3_train,epochs=30,shuffle=True)

Epoch 1/30
978/978 [==============================] - 1s 897us/step - loss: nan - acc: 0.0010
Epoch 2/30
978/978 [==============================] - 0s 184us/step - loss: nan - acc: 0.0010 
Epoch 3/30
978/978 [==============================] - 0s 182us/step - loss: nan - acc: 0.0010
Epoch 4/30
978/978 [==============================] - 0s 181us/step - loss: nan - acc: 0.0010 
Epoch 5/30
978/978 [==============================] - 0s 183us/step - loss: nan - acc: 0.0010 
Epoch 6/30
978/978 [==============================] - 0s 182us/step - loss: nan - acc: 0.0010 
Epoch 7/30
978/978 [==============================] - 0s 186us/step - loss: nan - acc: 0.0010     
Epoch 8/30
978/978 [==============================] - 0s 185us/step - loss: nan - acc: 0.0010   
Epoch 9/30
978/978 [==============================] - 0s 188us/step - loss: nan - acc: 0.0010 
Epoch 10/30
978/978 [==============================] - 0s 182us/step - loss: nan - acc: 0.0010 
Epoch 11/30
978/978 [========================